# Rent Division

In the paper [Which Is the Fairest (Rent Division) of Them All?
](http://procaccia.info/papers/rent.pdf), it was suggested that the popular envy-free model is insufficient. Below we presnet an initial implmentation of the envy-free model.

## Outline of the algorithm

1. Use linear programming to find a welfare-maximizing allocation $x$.
2. Use linear programming to find a price assignment $p$ that maximizes the minimum utility of each agent.

## 1. Use linear programming to find a welfare-maximizing allocation $x$.

Here is the following variables:

* Let $x_{ij}$ be the binary variable that suggests assignment of agent $i$ for room $j$

Above are the varaibles we want to know at the end of model optimization. Furthermore, we have **constants** $v_{ij}$ that stands for the private evaluation of room $j$ for agent $i$.

In [1]:
from gurobipy import Model, GRB, quicksum
import numpy as np

# create a LP model
model = Model()

# input parameters
v = np.array([
    [3, 0, 0],
    [0, 3, 0],
    [0, 0, 3]    
])
rent = v[0].sum()

# variables
n = range(3)
x = {}
for i in n:
    for j in n:
        x[str([i, j])] = model.addVar(
            vtype=GRB.BINARY, 
            name=str([i, j])
        )

model.update()

print(x)

{'[0, 0]': <gurobi.Var [0, 0]>, '[0, 1]': <gurobi.Var [0, 1]>, '[0, 2]': <gurobi.Var [0, 2]>, '[1, 0]': <gurobi.Var [1, 0]>, '[1, 1]': <gurobi.Var [1, 1]>, '[1, 2]': <gurobi.Var [1, 2]>, '[2, 0]': <gurobi.Var [2, 0]>, '[2, 1]': <gurobi.Var [2, 1]>, '[2, 2]': <gurobi.Var [2, 2]>}


### Objective

We want to maximize the social welfare

$$ \max \sum x_{ij} v_{ij}$$


In [2]:
welfare = []
for i in n:
    for j in n:
        welfare += [x[str([i, j])] * v[i,j]]
model.setObjective(
    quicksum(welfare),
    GRB.MAXIMIZE
)

### Constraints

* $\sum x_{ij} = 1, \forall i$ (Each agent can be assigned exactly one room)

In [3]:
for i in n:
    # c1: each agent must be assigned exactly one room
    model.addConstr(
        quicksum(x[str([i, j])] for j in n) == 1
    )
model.update()
model.optimize()
print()
print("*****Thus the objective value is " + str(model.ObjVal))

Optimize a model with 3 rows, 9 columns and 9 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 9 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+00, best bound 9.000000000000e+00, gap 0.0000%

*****Thus the objective value is 9.0


## Extracting the allocation

The following is the allocation:

In [4]:
x

{'[0, 0]': <gurobi.Var [0, 0] (value 1.0)>,
 '[0, 1]': <gurobi.Var [0, 1] (value 0.0)>,
 '[0, 2]': <gurobi.Var [0, 2] (value 0.0)>,
 '[1, 0]': <gurobi.Var [1, 0] (value 0.0)>,
 '[1, 1]': <gurobi.Var [1, 1] (value 1.0)>,
 '[1, 2]': <gurobi.Var [1, 2] (value 0.0)>,
 '[2, 0]': <gurobi.Var [2, 0] (value 0.0)>,
 '[2, 1]': <gurobi.Var [2, 1] (value 0.0)>,
 '[2, 2]': <gurobi.Var [2, 2] (value 1.0)>}

Let constants $\sigma_i = j$ stands for the room allocation of agent $i$ is $j$.

In [5]:
sigma = {}
for i in n:
    for j in n:
        if x[str([i, j])].X != 0:
            sigma[i] = j
sigma

{0: 0, 1: 1, 2: 2}

### (Digression) Let's just find one envy-free solution that doesn't maximize the minimum of agents' utility

Here is the following variables:

* Let $p_{ij}$ be the continuous variable that suggests payment of agent $i$ for room $j$

In [6]:
# variables
model = Model()
p = {}
for i in n:
    for j in n:
        p[str([i, j])] = model.addVar(
            vtype=GRB.CONTINUOUS, 
            name=str([i, j])
        )

model.update()

print(p)

{'[0, 0]': <gurobi.Var [0, 0]>, '[0, 1]': <gurobi.Var [0, 1]>, '[0, 2]': <gurobi.Var [0, 2]>, '[1, 0]': <gurobi.Var [1, 0]>, '[1, 1]': <gurobi.Var [1, 1]>, '[1, 2]': <gurobi.Var [1, 2]>, '[2, 0]': <gurobi.Var [2, 0]>, '[2, 1]': <gurobi.Var [2, 1]>, '[2, 2]': <gurobi.Var [2, 2]>}


### Objective

We create a dummy objective to get a feasible solution

$$\max \sum p_{ij}$$

In [7]:
dummy_objective = [p[str([i, j])] for i in n for j in n]
model.setObjective(
    quicksum(dummy_objective),
    GRB.MAXIMIZE
)

### Constraints

* $\sum p_{ij} = rent, \forall i,j$ (Agent's payments sum up to rent)
* $p_{ij} \geq 0, \forall i,j$ (Non-negative payments)
* $v_{i\sigma_i} - p_{i\sigma_i} \geq v_{ij} - p_{ij}, i \in N, j \in N$ (Agent's utility for the selected room is at least as high as the utility for any other room)

In [8]:
c = {}
# c1: payment must add up to rent
model.addConstr(
    quicksum(p[str([i, j])] for i in n for j in n) == rent
)
for i in n:
    for j in n:
        # c2: all payments are nonnegative
        model.addConstr(
            p[str([i, j])] >= 0
        )
        # c3: non-envy
        model.addConstr(
            (v[i,sigma[i]] - p[str([i, sigma[i]])]) >= (v[i,j] - p[str([i, j])])
        )

model.update()
model.optimize()
print()
print("*****Thus the objective value is " + str(model.ObjVal))

Optimize a model with 19 rows, 9 columns and 30 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve removed 12 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 9 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+00   4.206800e+01   0.000000e+00      0s
       7    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds
Optimal objective  3.000000000e+00

*****Thus the objective value is 3.0


In [9]:
p

{'[0, 0]': <gurobi.Var [0, 0] (value 0.0)>,
 '[0, 1]': <gurobi.Var [0, 1] (value 0.0)>,
 '[0, 2]': <gurobi.Var [0, 2] (value 0.0)>,
 '[1, 0]': <gurobi.Var [1, 0] (value 0.0)>,
 '[1, 1]': <gurobi.Var [1, 1] (value 0.0)>,
 '[1, 2]': <gurobi.Var [1, 2] (value 0.0)>,
 '[2, 0]': <gurobi.Var [2, 0] (value 0.0)>,
 '[2, 1]': <gurobi.Var [2, 1] (value 0.0)>,
 '[2, 2]': <gurobi.Var [2, 2] (value 3.0)>}

### 2. Use linear programming to find a price assignment $p$ that maximizes the minimum utility of each agent.

Here is the following variables:

* Let $p_{ij}$ be the continuous variable that suggests payment of agent $i$ for room $j$
* Let $R$ be the continuous variable that stands for the minimum of agents' utilities

In [10]:
# variables
model = Model()
R = model.addVar(
    vtype=GRB.CONTINUOUS, 
    name="R"
)
p = {}
for i in n:
    for j in n:
        p[str([i, j])] = model.addVar(
            vtype=GRB.CONTINUOUS, 
            name=str([i, j])
        )

model.update()

### Objective

We want to maximize the minimum of agents' utility

$$\max R$$

In [11]:
model.setObjective(
    R,
    GRB.MAXIMIZE
)

### Constraints

* $\sum p_{ij} = rent, \forall i,j$ (Agent's payments sum up to rent)
* $R \leq v_{i\sigma_i} - p_{i\sigma_i}, i \in N$ (R is smaller each agent's utility)
* $p_{ij} = 0, j \in N - \{\sigma_i\}$ (If agent $i$ is assigned to room $j$, she cannot pay for any other room)
* $p_{ij} \geq 0, \forall i,j$ (Non-negative payments)
* $v_{i\sigma_i} - p_{i\sigma_i} \geq v_{ij} - p_{ij}, i \in N, j \in N$ (Agent's utility for the selected room is at least as high as the utility for any other room)

In [12]:
# c1: payment must add up to rent
model.addConstr(
    quicksum(p[str([i, j])] for i in n for j in n) == rent
)
for i in n:
    # c2: R is smaller each agent's utility
    model.addConstr(
        R <= (v[i,sigma[i]] - p[str([i, sigma[i]])])
    )
    # c3: (If agent $i$ is assigned to room $j$, she cannot pay for any other room)
    room_not_assigned = [j for j in n if j !=sigma[i]]
    for j in room_not_assigned:
        model.addConstr(
            p[str([i, j])] == 0
        )
    for j in n:
        # c4: all payments are nonnegative
        model.addConstr(
            p[str([i, j])] >= 0
        )
        # c5: non-envy
        model.addConstr(
            (v[i,sigma[i]] - p[str([i, sigma[i]])]) >= (v[i,j] - p[str([i, j])])
        )

model.update()
model.optimize()
print()
print("*****Thus the objective value is " + str(model.ObjVal))

Optimize a model with 28 rows, 10 columns and 42 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve removed 24 rows and 6 columns
Presolve time: 0.01s
Presolved: 4 rows, 4 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+00   3.000000e+00   0.000000e+00      0s
       2    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  2.000000000e+00

*****Thus the objective value is 2.0


In [13]:
p

{'[0, 0]': <gurobi.Var [0, 0] (value 1.0)>,
 '[0, 1]': <gurobi.Var [0, 1] (value 0.0)>,
 '[0, 2]': <gurobi.Var [0, 2] (value 0.0)>,
 '[1, 0]': <gurobi.Var [1, 0] (value 0.0)>,
 '[1, 1]': <gurobi.Var [1, 1] (value 1.0)>,
 '[1, 2]': <gurobi.Var [1, 2] (value 0.0)>,
 '[2, 0]': <gurobi.Var [2, 0] (value 0.0)>,
 '[2, 1]': <gurobi.Var [2, 1] (value 0.0)>,
 '[2, 2]': <gurobi.Var [2, 2] (value 1.0)>}

## Complete standalone script

In [14]:
from gurobipy import Model, GRB, quicksum
import numpy as np

model = Model()
model.setParam('OutputFlag', False)

# ===========================================================
# Get the allocation first
# ===========================================================

# **Input parameters**
v = np.array([
    [3, 0, 0],
    [0, 3, 0],
    [0, 0, 3]    
])
print("The agents' values are \n", v)
rent = v[0].sum()

# **Variables**
n = range(3)
x = {}
for i in n:
    for j in n:
        x[str([i, j])] = model.addVar(
            vtype=GRB.BINARY, 
            name=str([i, j])
        )
model.update()

# **Objective**
welfare = []
for i in n:
    for j in n:
        welfare += [x[str([i, j])] * v[i,j]]
model.setObjective(
    quicksum(welfare),
    GRB.MAXIMIZE
)

# **Constraints**
for i in n:
    # c1: each agent must be assigned exactly one room
    model.addConstr(
        quicksum(x[str([i, j])] for j in n) == 1
    )
model.update()
model.optimize()
print("*****The maximum social welfare is " + str(model.ObjVal))

# Room assignment
sigma = {}
for i in n:
    for j in n:
        if x[str([i, j])].X != 0:
            sigma[i] = j
            print(f"agent {i} is assigned room {j}")
            
# ===========================================================
# Get the pricing
# ===========================================================

# **Variables**
model = Model()
model.setParam('OutputFlag', False)
R = model.addVar(
    vtype=GRB.CONTINUOUS, 
    name="R"
)
p = {}
for i in n:
    for j in n:
        p[str([i, j])] = model.addVar(
            vtype=GRB.CONTINUOUS, 
            name=str([i, j])
        )
model.update()

# **Objective**
model.setObjective(
    R,
    GRB.MAXIMIZE
)

# **Constraints**
# c1: payment must add up to rent
model.addConstr(
    quicksum(p[str([i, j])] for i in n for j in n) == rent
)
for i in n:
    # c2: R is smaller each agent's utility
    model.addConstr(
        R <= (v[i,sigma[i]] - p[str([i, sigma[i]])])
    )
    # c3: (If agent $i$ is assigned to room $j$, she cannot pay for any other room)
    room_not_assigned = [j for j in n if j !=sigma[i]]
    for j in room_not_assigned:
        model.addConstr(
            p[str([i, j])] == 0
        )
    for j in n:
        # c4: all payments are nonnegative
        model.addConstr(
            p[str([i, j])] >= 0
        )
        # c5: non-envy
        model.addConstr(
            (v[i,sigma[i]] - p[str([i, sigma[i]])]) >= (v[i,j] - p[str([i, j])])
        )
model.update()
model.optimize()
print("*****The minimum of agents' utility is " + str(model.ObjVal))

# price assignment
for i in n:
    for j in n:
        if p[str([i, j])].X != 0:
            print(f"agent {i} will pay {p[str([i, j])].X} for room {j}")

The agents' values are 
 [[3 0 0]
 [0 3 0]
 [0 0 3]]
*****The maximum social welfare is 9.0
agent 0 is assigned room 0
agent 1 is assigned room 1
agent 2 is assigned room 2
*****The minimum of agents' utility is 2.0
agent 0 will pay 1.0 for room 0
agent 1 will pay 1.0 for room 1
agent 2 will pay 1.0 for room 2
